In [9]:
LIST_OF_SUPERHEROES = [
    "Batman",
    "Superman",
    "Wonder Woman",
    "Spiderman",
    "Ironman",
    "Captain America",
    "Black Widow",
    "Hulk",
    "Thor",
    "Deadpool",
    "Star Lord",
    "Thanos",
    "Groot",
    "Rocket",
    "Doctor Strange",
    "Drax",
    "Vision",
    "Jarvis",
]

SUPERHERO_SYNONYMS = {
    "Batman": ["Bruce Wayne"],
    "Superman": ["Clark Kent"],
    "Wonder Woman": ["Diana Prince"],
    "Spiderman": ["Peter Parker"],
    "Ironman": ["Tony Stark"],
    "Captain America": ["Steve Rogers"],
    "Black Widow": ["Natasha Romanoff"],
    "Hulk": ["Bruce Banner"],
    "Thor": ["Thor Odinson"],
    "Deadpool": ["Wade Wilson"],
    "Star Lord": ["Peter Quill"],
    "Thanos": ["Thanos"],
    "Groot": ["Groot"],
    "Rocket": ["Rocket"],
    "Doctor Strange": ["Stephen Strange"],
    "Drax": ["Drax"],
    "Vision": ["Vision"],
    "Jarvis": ["Jarvis"],
}

In [4]:
from os.path import join as pjoin

In [5]:
root = '/home/quamer23nasim38/Superhero-Character-Based-On-RAG-AI-Using-Haystack-And-Qdrant'
data_folder = 'data'

In [6]:
import os

In [7]:
all_movie_scripts = os.listdir(pjoin(root, data_folder))

In [8]:
for superhero in LIST_OF_SUPERHEROES:
    print(f'Processing {superhero}')
    superhero_script = []
    for script in all_movie_scripts:
        

['spider-man-no-way-home-2021.pdf',
 'avengers infinity war.pdf',
 'thor-ragnorak-2017.pdf',
 'iron-man-2008.pdf',
 'Captain America: Civil War.pdf',
 'Man of Steel.pdf',
 'the-avengers-2012.pdf',
 'Avengers: Age of Ultron.txt',
 'deadpool-2016.pdf',
 'captain america winter solider.pdf',
 'Guardians of the Galaxy vol 2.pdf',
 'thor-2011.pdf',
 'Guardians_of_the_Galaxy_Movie_Transcript.pdf',
 'the-dark-knight-2008.pdf',
 'avengers-endgame.pdf',
 'Batman_VS_Superman.pdf',
 'batman-begins-2005.pdf',
 'Batman-v-Superman-Dawn-of-Justice.pdf',
 'Deadpool 2.txt',
 'wonder-woman-2017.pdf',
 'the-dark-knight-rises-2012.pdf',
 'captain-america-the-first-avenger-20.pdf']